In [1]:
import argparse
import numpy as np
import pandas as pd
import ast
from pycaret.regression import *
from utils import preprocess


In [2]:
train_data = preprocess(pd.read_csv("train.tsv", sep="\t"))
test_data = preprocess(pd.read_csv("test.tsv", sep="\t"))

In [3]:
train_data

,belongs_to_collection,budget,id,original_language,release_date,revenue,runtime,vote_average,vote_count,release_year,release_month,director,Western,Horror,Thriller,Crime,History,Music,War,TV Movie,Action,Documentary,Drama,Adventure,Mystery,Family,Comedy,Fantasy,Animation,Science Fiction,Romance
0,"{'id': 556, 'name': 'Spider-Man Collection', '...",258000000.0,559,en,2007-05-01,890871626,139.0,6.2,8180,2007,5,Sam Raimi,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0
1,"{'id': 256296, 'name': 'Silent Night, Deadly N...",250000.0,50719,en,1987-04-10,154323,88.0,4.3,68,1987,4,Lee Harry,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,{},NaN,469062,en,2018-07-27,176236,98.0,6.5,15,2018,7,Matt Tyrnauer,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,"{'id': 8917, 'name': 'Hellraiser Collection', ...",1000000.0,9003,en,1987-09-11,14564027,94.0,6.9,1115,1987,9,Clive Barker,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"{'id': 108693, 'name': 'National Lampoon's Vac...",15000000.0,11153,en,1983-07-28,61399552,99.0,7.1,782,1983,7,Harold Ramis,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5210,"{'id': 239968, 'name': 'La cage aux folles Col...",NaN,78159,fr,1985-11-20,345280,87.0,4.7,29,1985,11,Georges Lautner,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
5211,"{'id': 111751, 'name': 'Texas Chainsaw Massacr...",85000.0,30497,en,1974-10-01,30859000,83.0,7.3,1483,1974,10,Tobe Hooper,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5212,{},30000000.0,14342,en,1998-05-29,6100000,90.0,5.6,91,1998,5,Christopher Guest,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0
5213,{},20000000.0,38,en,2004-03-19,72258126,108.0,8.1,8753,2004,3,Michel Gondry,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1


In [4]:
pycar = setup(data=train_data, test_data=test_data, target='revenue', normalize=True, ignore_features = ["id"])


,Description,Value
0,session_id,8803
1,Target,revenue
2,Original Data,"(5215, 31)"
3,Missing Values,True
4,Numeric Features,5
5,Categorical Features,23
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(5215, 3372)"


In [5]:
lbgm = create_model("lightgbm")

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,31130612.4470,5120365684246974.0000,71556730.5307,0.7897,2.1190,4586.9743
1,27295484.5304,4684647213067871.0000,68444482.7073,0.7148,2.4759,9375.0325
2,26202480.2868,2947644240727690.0000,54292211.6028,0.8212,2.4844,13599.5167
3,32233979.2799,6072511983686170.0000,77926324.0740,0.8152,2.5909,16298.2484
4,29391233.6007,4173689350527758.5000,64604097.0104,0.8225,2.2855,6956.9876
5,31356354.6139,5616063984454774.0000,74940402.8843,0.7819,2.5858,19324.5964
6,28640737.9676,3517758515881423.0000,59310694.7850,0.8032,2.6012,19113.3224
7,31907430.9333,4427553875272565.5000,66539866.8114,0.6148,2.6789,8415.7030
8,26637989.5572,2457988851080351.0000,49578108.5872,0.8649,2.7537,48138.1327
9,26241556.0638,2238094255134716.2500,47308500.8760,0.8742,2.5487,31320.1158


In [12]:
tuned_lgbm = tune_model(lbgm, optimize = "RMSLE",fold = 5, n_iter=200)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,31290905.7422,5072556657551785.0000,71221883.2772,0.7517,2.2910,10057.1625
1,32590330.1879,6734286605547016.0000,82062699.2339,0.7276,2.5316,14226.3937
2,31868739.2609,5379608913227824.0000,73345817.2852,0.7815,2.3680,14843.0094
3,30993553.4908,4560445550021622.0000,67531070.9971,0.6892,2.5668,6763.2934
4,28424546.0258,3035298729459007.0000,55093545.2613,0.8314,2.5603,25416.2189
Mean,31033614.9415,4956439291161451.0000,69851003.2109,0.7563,2.4635,14261.2156
SD,1413919.3096,1200074076540735.0000,8790713.3719,0.0482,0.1127,6303.4387


In [7]:
extrees = create_model("et")

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,31598817.7948,5105404236920050.0000,71452111.4938,0.7903,1.8298,2707.7654
1,26997718.2195,4564023459383069.0000,67557556.6416,0.7222,2.0930,9060.3301
2,26786487.9917,3810190423549021.5000,61726739.9394,0.7689,2.3461,17492.2419
3,31869052.1942,7273585192502692.0000,85285316.3945,0.7786,2.4381,10566.3760
4,29646473.8292,4378689224762979.0000,66171664.8178,0.8138,2.0600,3853.5690
5,32944866.4612,7442620957487414.0000,86270626.2727,0.7110,2.2545,7455.8222
6,28116326.6103,4207331467035868.5000,64863945.8177,0.7646,2.2165,2786.2159
7,29821694.2840,3399195366183564.5000,58302618.8621,0.7043,2.4048,7175.9572
8,27828034.6785,3085867709741713.0000,55550586.9433,0.8304,2.4508,22922.9496
9,25493138.6126,2419113019690815.0000,49184479.4594,0.8640,2.3220,16361.5852


In [10]:
tuned_extrees = tune_model(extrees, optimize="RMSLE", fold = 3)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,30009484.6534,4815252305156212.0000,69392019.0307,0.7512,2.3224,10112.4549
1,32971611.5631,7081113562196583.0000,84149352.7141,0.7372,2.4081,5803.0085
2,27818358.7846,2935083208113323.0000,54176408.2246,0.8061,2.5413,16529.3434
Mean,30266485.0003,4943816358488707.0000,69239259.9898,0.7648,2.4240,10814.9356
SD,2111640.8207,1695049352497226.7500,12236880.0990,0.0297,0.0900,4407.0908


In [5]:
top3 = compare_models(n_select = 3, fold=3, sort = 'RMSLE')
print(top3)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,29648317.1996,4773051742556794.0000,68411519.0325,0.7691,2.2513,11910.1875,15.6667
rf,Random Forest Regressor,29613701.4445,4798925200807970.0000,68398589.2135,0.7701,2.3173,13179.8427,8.5867
lightgbm,Light Gradient Boosting Machine,30210433.2018,4644746286473126.0000,67354505.7294,0.7769,2.5150,17374.0971,0.4400
gbr,Gradient Boosting Regressor,32982368.3570,4874231686640837.0000,69232890.1959,0.7632,2.7950,27834.5725,4.6700
knn,K Neighbors Regressor,36233293.3333,5500701249634304.0000,73354966.6667,0.7350,2.8291,45695.0885,8.1500
omp,Orthogonal Matching Pursuit,40545405.1581,7486594330471352.0000,85808574.0653,0.6361,2.9547,45575.3285,0.4767
huber,Huber Regressor,51213330.6149,16101866900071098.0000,125700841.4525,0.2207,3.0139,45597.0415,2.1300
par,Passive Aggressive Regressor,60878133.6417,23076869126786372.0000,150851858.2077,-0.1270,3.0294,26882.5912,21.9200
dt,Decision Tree Regressor,38447258.2501,7745883332847344.0000,87535648.3706,0.6184,3.0717,8579.1187,0.3433
llar,Lasso Least Angle Regression,89054725952.4445,41261220872853539384197120.0000,3708650575434.8418,-1531450232.2142,3.0913,57909.5416,0.8267


[ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=100, n_jobs=-1, oob_score=False,
                    random_state=8803, verbose=0, warm_start=False), RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=8803, verbose=0, warm_start=False), LGBMRegressor(boosti

In [7]:
for model, name in zip(top3, ["extrees", "randomforest", "lgbm"]):
    save_model(model, f'{name}')


Transformation Pipeline and Model Succesfully Saved
Transformation Pipeline and Model Succesfully Saved
Transformation Pipeline and Model Succesfully Saved


In [8]:
tuned_extrees = tune_model(top3[0], fold=3, n_iter=15)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,29420824.4014,4520741786004795.0000,67236461.7303,0.7664,2.3877,10408.4497
1,33201109.1396,6868134037830827.0000,82874206.1068,0.7451,2.5033,6181.1606
2,28455269.2388,2990760663824958.0000,54687847.4967,0.8024,2.6342,16187.0039
Mean,30359067.5933,4793212162553527.0000,68266171.7779,0.7713,2.5084,10925.5381
SD,2047921.5978,1594613046046986.0000,11530045.6782,0.0237,0.1007,4101.1999
